In [2]:
import os
import yaml
import json
import pandas as pd

In [3]:
%load_ext autoreload

In [4]:
%autoreload 2

In [5]:
import wombat.bsub as bsub

load sample data

In [9]:
m = yaml.safe_load(open('../tests/data/bulk_expression/C3L-00677.yaml'))
m

{'id': 'C3L-00677',
 'rna-seq': {'tumor': {'R1': {'filepath': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/test_samples/C3L-00677/rna-seq/170906_UNC32-K00270_0058_BHLJ7FBBXX_TGACCA_S4_L007_R1_001.fastq.gz'},
   'R2': {'filepath': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/test_samples/C3L-00677/rna-seq/170906_UNC32-K00270_0058_BHLJ7FBBXX_TGACCA_S4_L007_R2_001.fastq.gz'}}},
 'wxs': {'normal': {'barcode': 'CPT0071960002',
   'filepath': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/test_samples/C3L-00677/wxs/5cee818c-61e8-4441-bd9a-5c984cde9c6b_gdc_realn.bam'},
  'tumor': {'barcode': 'CPT0071100006',
   'filepath': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/test_samples/C3L-00677/wxs/1d6c4f0f-09c4-4472-a19e-30ee7e0ecf64_gdc_realn.bam'}}}

make test input yaml

In [12]:
template = yaml.safe_load(open('/home/estorrs/pecgs-pipeline/submodules/pecgs-bulk-expression/cwl/template.bulk_expression.yaml'))
template


{'cpu': 'a_int',
 'fq_1': [{'class': 'File', 'path': 'a/file/path'}],
 'fq_2': [{'class': 'File', 'path': 'a/file/path'}],
 'gene_info': {'class': 'File', 'path': 'a/file/path'},
 'gtf': {'class': 'File', 'path': 'a/file/path'},
 'star_index': {'class': 'Directory', 'path': 'a/dir/path'}}

In [13]:
template['star_index']['path'] = '/scratch1/fs1/dinglab/estorrs/pecgs_resources/bulk_expression/STAR_GRCh38_d1_vd1_d1_GENCODE_v34'
template['gtf']['path'] = '/scratch1/fs1/dinglab/estorrs/pecgs_resources/bulk_expression/STAR_GRCh38_d1_vd1_d1_GENCODE_v34/gencode.v34.annotation.gtf'
template['gene_info']['path'] = '/scratch1/fs1/dinglab/estorrs/pecgs_resources/bulk_expression/STAR_GRCh38_d1_vd1_d1_GENCODE_v34/gencode.gene.info.v34.tsv'

template['cpu'] = 20 
template['fq_1'][0]['path'] = m['rna-seq']['tumor']['R1']['filepath']
template['fq_2'][0]['path'] = m['rna-seq']['tumor']['R2']['filepath']


In [14]:
template

{'cpu': 20,
 'fq_1': [{'class': 'File',
   'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/test_samples/C3L-00677/rna-seq/170906_UNC32-K00270_0058_BHLJ7FBBXX_TGACCA_S4_L007_R1_001.fastq.gz'}],
 'fq_2': [{'class': 'File',
   'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/test_samples/C3L-00677/rna-seq/170906_UNC32-K00270_0058_BHLJ7FBBXX_TGACCA_S4_L007_R2_001.fastq.gz'}],
 'gene_info': {'class': 'File',
  'path': '/scratch1/fs1/dinglab/estorrs/pecgs_resources/bulk_expression/STAR_GRCh38_d1_vd1_d1_GENCODE_v34/gencode.gene.info.v34.tsv'},
 'gtf': {'class': 'File',
  'path': '/scratch1/fs1/dinglab/estorrs/pecgs_resources/bulk_expression/STAR_GRCh38_d1_vd1_d1_GENCODE_v34/gencode.v34.annotation.gtf'},
 'star_index': {'class': 'Directory',
  'path': '/scratch1/fs1/dinglab/estorrs/pecgs_resources/bulk_expression/STAR_GRCh38_d1_vd1_d1_GENCODE_v34'}}

In [15]:
yaml.safe_dump(template, open('../tests/data/bulk_expression/C3L-00677_inputs.yaml', 'w'))

make cromwell compute1 config

In [37]:
args = bsub.DEFAULT_ARGS
args['group_name'] = 'test_cromwell'
args

{'mem': 10,
 'n_processes': 1,
 'max_mem': None,
 'docker': 'python:3.8',
 'queue': 'dinglab',
 'group': 'compute-dinglab',
 'group_name': 'test_cromwell',
 'n_concurrent': 10,
 'interactive': False,
 'username': 'estorrs'}

In [43]:
workflow_root = '/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/bulk_expression/C3L-00677'
output_fp = '../tests/data/bulk_expression/C3L-00677.cromwell-config-db.compute1.dat'
args['queue'] = 'general'
bsub.save_compute1_cromwell_template(workflow_root, output_fp, queue=args['queue'])

make execution script

In [44]:
TEST_DIR_ROOT = '/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/bulk_expression'
TOOL_ROOT = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline'

dconfig = os.path.join(TEST_DIR_ROOT, 'C3L-00677.cromwell-config-db.compute1.dat')
cwl_fp = os.path.join(TOOL_ROOT, 'submodules', 'pecgs-bulk-expression', 'cwl', 'bulk_expression.cwl')
inputs_fp = os.path.join(TEST_DIR_ROOT, 'C3L-00677_inputs.yaml')
volumes = [TEST_DIR_ROOT, TOOL_ROOT, workflow_root, '/storage1/fs1/dinglab/Active', '/storage1/fs1/m.wyczalkowski/Active', '/scratch1/fs1/dinglab/estorrs']

In [45]:
start_commands, run_command = bsub.cromwell_commands(dconfig, cwl_fp, inputs_fp, args, volumes,
                                                     workflow_root=workflow_root)
filepath = os.path.join('../tests/data/bulk_expression/start_server.sh')
bsub.write_command_file(start_commands, filepath)
filepath = os.path.join('../tests/data/bulk_expression/run_job.sh')
bsub.write_command_file([run_command], filepath)

In [46]:
for c in start_commands: print(c)

mkdir -p /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/bulk_expression/C3L-00677/cromwell-workdir/logs
source /opt/ibm/lsfsuite/lsf/conf/lsf.conf
export LSF_DOCKER_NETWORK=host
export LSF_DOCKER_VOLUMES="/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/bulk_expression:/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/bulk_expression /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline:/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/bulk_expression/C3L-00677:/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/bulk_expression/C3L-00677 /storage1/fs1/dinglab/Active:/storage1/fs1/dinglab/Active /storage1/fs1/m.wyczalkowski/Active:/storage1/fs1/m.wyczalkowski/Active /scratch1/fs1/dinglab/estorrs:/scratch1/fs1/dinglab/estorrs"
bgadd -L 10 /estorrs/test_cromwell
bsub -n 1 -q general-interactive -G compute-dinglab -a 'docker(mwyczalkowski/cromwell-runner)' -g /estorrs/t

In [47]:
print(run_command)

/usr/bin/java -Dconfig.file=/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/bulk_expression/C3L-00677.cromwell-config-db.compute1.dat -Djavax.net.ssl.trustStorePassword=changeit -Djavax.net.ssl.trustStore=/gscmnt/gc2560/core/genome/cromwell/cromwell.truststore -jar /usr/local/cromwell/cromwell-47.jar run -t cwl -i /storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/bulk_expression/C3L-00677_inputs.yaml /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline/submodules/pecgs-bulk-expression/cwl/bulk_expression.cwl
